# Testing

In [1]:
!pip install datasets peft bitsandbytes safetensors scikit-learn
!pip install torch torchvision torchaudio
!pip install transformers
!pip install loguru
!pip install peft
!pip install tensorboard
!pip install sentencepiece
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 10.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of

### Load Model
Load the base model and finetuned model

In [2]:
from sklearn.metrics import accuracy_score,f1_score
from datasets import load_dataset
from peft import PeftModel
from tqdm import tqdm
import pandas as pd
import datasets
import torch
from transformers import (
    AutoModel,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
    LlamaForCausalLM
)

base_model = "daryl149/llama-2-13b-chat-hf"
peft_model = "/workspace/Llama-3.1-fin-output"

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
model = LlamaForCausalLM.from_pretrained(
    base_model,
    # load_in_8bit=True,
    # load_in_4bit=True,
    trust_remote_code=True,
    device_map='auto',
    torch_dtype=torch.float16,
)
model = PeftModel.from_pretrained(model, peft_model)

model = model.eval()
tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


...

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Some parameters are on the meta device because they were offloaded to the cpu.


In [3]:

def test(model, tokenizer, batch_size = 6, prompt_fun = None ):
    dataset = load_dataset('TheFinAI/flare-cfa')
    dataset = dataset['test']
    dataset = dataset.to_pandas()
    dataset = dataset.head(50)

    # print example
    print(f"\n\nPrompt example:\n{dataset['text'][0]}\n\n")

    context = dataset['text'].tolist()
    context = [f"""
    Instruction: What is the question to the answer? Choose from A, B, C. \nInput: {x}\nAnswer:""" for x in context]
    total_steps = dataset.shape[0] // batch_size + 1
    print(f"Total len: {len(context)}. Batchsize: {batch_size}. Total steps: {total_steps}")


    out_text_list = []
    for i in tqdm(range(total_steps)):
        tmp_context = context[i* batch_size: (i+1)* batch_size]
        tokens = tokenizer(tmp_context, return_tensors='pt', padding=True, max_length=512)
        for k in tokens.keys():
            tokens[k] = tokens[k].cuda()
        res = model.generate(**tokens, max_length=512)
        res_sentences = [tokenizer.decode(i) for i in res]
        out_text = [o.split("Answer:  ")[1][0] for o in res_sentences]
        print(out_text)
        out_text_list += out_text
        torch.cuda.empty_cache()

    dataset["out_text"] = out_text_list
    # dataset["new_target"] = dataset["target"].apply(change_target)
    # dataset["new_out"] = dataset["out_text"].apply(change_target)
    

    acc = accuracy_score(dataset["answer"], dataset["out_text"])
    f1_macro = f1_score(dataset["new_target"], dataset["new_out"], average = "macro")
    f1_micro = f1_score(dataset["new_target"], dataset["new_out"], average = "micro")
    f1_weighted = f1_score(dataset["new_target"], dataset["new_out"], average = "weighted")

    print(f"Acc: {acc}. F1 macro: {f1_macro}. F1 micro: {f1_micro}. F1 weighted (BloombergGPT): {f1_weighted}. ")
    print(f"Acc: {acc}. ")

    return dataset

dataset = test(model, tokenizer)

README.md:   0%|          | 0.00/487 [00:00<?, ?B/s]

(…)-00000-of-00001-6e96e3a2bc65fdb8.parquet:   0%|          | 0.00/285k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1032 [00:00<?, ? examples/s]



Prompt example:
Q:The nominal risk-free rate is best described as the sum of the real risk-free rate and a premium for:,CHOICES: A: maturity.,B: liquidity.,C: expected inflation.


Total len: 50. Batchsize: 6. Total steps: 9


  0%|          | 0/9 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2855: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
  0%|          | 0/9 [03:30<?, ?it/s]


KeyboardInterrupt: 

### Testing code

You can follow one of the files here: https://github.com/AI4Finance-Foundation/FinNLP/tree/main/finnlp/benchmarks

In [ ]:

def test(model, tokenizer, batch_size = 6, prompt_fun = None ):
    dataset = load_dataset('TheFinAI/flare-cfa')
    dataset = dataset['test']
    dataset = dataset.to_pandas()
    dataset = dataset.head(50)

    # print example
    print(f"\n\nPrompt example:\n{dataset['text'][0]}\n\n")

    context = dataset['text'].tolist()
    context = [f"""
    Instruction: What is the question to the answer? Choose from A, B, C. \nInput: {x}\nAnswer:""" for x in context]
    total_steps = dataset.shape[0] // batch_size + 1
    print(f"Total len: {len(context)}. Batchsize: {batch_size}. Total steps: {total_steps}")


    out_text_list = []
    for i in tqdm(range(total_steps)):
        tmp_context = context[i* batch_size: (i+1)* batch_size]
        tokens = tokenizer(tmp_context, return_tensors='pt', padding=True, max_length=512)
        for k in tokens.keys():
            tokens[k] = tokens[k].cuda()
        res = model.generate(**tokens, max_length=512)
        res_sentences = [tokenizer.decode(i) for i in res]
        out_text = [o.split("Answer:  ")[1][0] for o in res_sentences]
        print(out_text)
        out_text_list += out_text
        torch.cuda.empty_cache()

    dataset["out_text"] = out_text_list
    # dataset["new_target"] = dataset["target"].apply(change_target)
    # dataset["new_out"] = dataset["out_text"].apply(change_target)
    

    acc = accuracy_score(dataset["answer"], dataset["out_text"])
    f1_macro = f1_score(dataset["new_target"], dataset["new_out"], average = "macro")
    f1_micro = f1_score(dataset["new_target"], dataset["new_out"], average = "micro")
    f1_weighted = f1_score(dataset["new_target"], dataset["new_out"], average = "weighted")

    print(f"Acc: {acc}. F1 macro: {f1_macro}. F1 micro: {f1_micro}. F1 weighted (BloombergGPT): {f1_weighted}. ")
    print(f"Acc: {acc}. ")

    return dataset

dataset = test(model, tokenizer)

README.md:   0%|          | 0.00/487 [00:00<?, ?B/s]

(…)-00000-of-00001-6e96e3a2bc65fdb8.parquet:   0%|          | 0.00/285k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1032 [00:00<?, ? examples/s]



Prompt example:
Q:The nominal risk-free rate is best described as the sum of the real risk-free rate and a premium for:,CHOICES: A: maturity.,B: liquidity.,C: expected inflation.


Total len: 50. Batchsize: 6. Total steps: 9


  0%|          | 0/9 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2855: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
